### Distance analysis of the testing data on the training data

In [25]:
import sys
sys.path.append("./src") # append to system path

from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import cross_validation

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.patches import Rectangle
style.use('ggplot')

/home/runsheng/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [26]:
train = pd.read_csv('../data/descs/train/descs_Mar08_3839_train.csv',header=0).fillna(0).values
test = pd.read_csv('../data/descs/test/descs_Mar08_3839_test.csv',header=0).fillna(0).values

In [28]:
train,val = cross_validation.train_test_split(train, test_size=0.1, random_state=3)

### Do Normalization and PCA

In [30]:
this_scaler = StandardScaler()
pca = PCA(n_components = 60)

train = this_scaler.fit_transform(train)
val = this_scaler.transform(val)
test = this_scaler.transform(test)

train = pca.fit_transform(train)
val = pca.transform(val)
test = pca.transform(test)

(140, 60) (10, 60)


In [31]:
print train.shape, val.shape, test.shape

(140, 60) (16, 60) (10, 60)


In [32]:
center = np.mean(train,axis=0)
print center

[ -1.94130426e-15   8.37425367e-16  -2.09356342e-16   1.70974346e-15
  -5.67799775e-16  -5.86832170e-17  -6.02692499e-16   2.03012210e-16
   6.18552828e-17   5.99520433e-16  -2.22044605e-16   4.82153999e-16
   5.07530526e-17  -1.90323947e-17   7.93016446e-17  -1.19745483e-16
  -1.05471187e-16   3.48927236e-17  -3.40997072e-16   7.23230999e-16
   2.12528408e-16   4.21884749e-16   9.19899078e-17   1.03092138e-16
  -6.02692499e-17  -3.00553233e-16   8.40597433e-17  -1.83979816e-16
  -9.67480064e-17   2.64867493e-16  -5.30528002e-16  -2.77952264e-16
  -1.87151881e-16   4.15540618e-16   3.48927236e-17  -4.06817437e-16
   4.52019374e-17   4.12368552e-17   5.69385808e-16  -2.52972246e-16
   1.64947421e-16   1.46708043e-17   1.23710566e-16  -3.14034513e-16
  -1.22124533e-16  -2.12528408e-16   3.01346250e-16   3.56857401e-16
   2.00236653e-17  -1.90323947e-17  -3.61615499e-16  -9.40963577e-17
  -8.64387926e-17   1.45122010e-16   2.06184276e-17   1.31640730e-16
  -3.19585628e-16   2.15700473e-16

### Determine the cut-off threshold on the validation set

In [34]:
val_pred = [4.7562,0.851,1.251,0.493,0.533,0.3928,3.9012,1.4881,0.4772,0.8184,0.9817,0.8649,1.0545,0.3667,0.5963,1.4904]
val_y = [3.982,0.4371,1.3205,2.1367,0.9894,0.2172,3.3901,1.3198,0.5471,1.6009,0.8481,2.0281,1.0528,0.3314,1.1539,1.6045]

distance = []
for each_tst in test:
    e_d = np.linalg.norm(each_tst-center)
    distance.append(e_d)
    
for i in distance:
    print i


30.3000035024
23.1525603706
26.6724463103
82.3957777458
63.3553663628
26.9331691107
38.8064541277
33.1876072133
33.441636499
38.8984038147
21.9824119069
46.6068414668
26.5092771699
20.4484746009
28.4736935574
33.1998629162


In [35]:
def mre(true_y,pred_y):
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.abs(np.subtract(true_y, pred_y) / true_y) * 100

val_rel_error = mre(val_y,val_pred)

for each_pair in zip(val_rel_error, distance):
    print each_pair
print r2_score(val_rel_error,distance) 

(19.442491210446999, 30.300003502396955)
(94.69229009380004, 23.152560370575017)
(5.2631578947368505, 26.67244631026292)
(76.927037019703278, 82.395777745771838)
(46.128967050737813, 63.355366362843078)
(80.847145488029454, 26.933169110734532)
(15.076251438010676, 38.80645412772526)
(12.751932110925889, 33.18760721333549)
(12.776457685980628, 33.44163649904096)
(48.878755699918798, 38.898403814716445)
(15.752859332625876, 21.982411906907398)
(57.354173857304858, 46.606841466782321)
(0.16147416413374191, 26.509277169926499)
(10.651780325890179, 20.448474600874491)
(48.323078256348026, 28.473693557394483)
(7.1112496104705576, 33.199862916197276)
0.102718334953


In [43]:
def distance_analysis(distance, error, thre_dist=30):
    #cut off the error by the threshold distance
    outside = []
    inside = []
    for each_dist, each_error in zip(distance, error):
        if each_dist <= thre_dist:
            inside.append(each_error)
        else:
            outside.append(each_error)
    return np.mean(inside),np.mean(outside)

inside_mean, outside_mean = distance_analysis(distance,val_rel_error,70)
print inside_mean, outside_mean

31.6808042813 76.9270370197


### Decide whether each testing chemical in our outside AD using the determined threshold distance

using 70

In [45]:
test_pred = [1.5622,1.2799,2.3694,3.8848,3.0489,0.8375,3.0248,0.8014,1.1162,0.4730]
test_y = [1.3169,1.3793,1.3008,6.8024,4.6775,0.6700,2.0106,0.2435,1.0148,0.4952]
test_dist = []

for each_tst in test:
    e_d = np.linalg.norm(each_tst-center)
    test_dist.append(e_d)
    
test_rel_error = mre(test_y,test_pred)

In [53]:
test_inside, test_outside = [],[]
for each_test_dist,each_test_error in zip(test_dist,test_rel_error):
    if each_test_dist <= 70:
        
        test_inside.append(each_test_error)
    else:
        test_outside.append(each_test_error)

print np.mean(test_inside)
print np.mean(test_outside)

46.9529969605
82.1494464945
